In [69]:
import sqlite3
import pandas as pd
import pdfkit
conn = sqlite3.connect("..//sqlite_DB//Movie_report.db")
cur = conn.cursor()
select_movie_fact ="""with last_six_months as (
select Month from DateDim where Month <=200009 order by Month desc limit 6),
max_review_count as (
select 
max(a.Rating_monthly_count)/2  max_count,
Review_month from 
  Movie_ratings_monthly a,last_six_months b
where a.Review_month =b.Month
group by Review_month),
movie_six_months as
(select a.MovieID,
a.Rating_monthly_count,
a.Review_month,
a.Rating_monthly_avg,
a.OverallCount,
a.Overall_rating_avg,
row_number() over(PARTITION by a.Review_month order by Rating_monthly_avg desc) rnk
from
Movie_ratings_monthly a,max_review_count c
where a.Review_month=c.Review_month
and a.Rating_monthly_count >= c.max_count)
select
a.MovieID, 
a.rnk ,
b.Title,
a.Rating_monthly_count,
a.Review_month,
a.Rating_monthly_avg,
a.OverallCount,
a.Overall_rating_avg
 from movie_six_months a,MovieDIM b
 where a.MovieID =b.MovieID and 
  a.rnk <= 15
  order by b.Title,a.Review_month desc
"""
df_details = pd.read_sql_query(select_movie_fact, conn)
conn.close()
df_details

,MovieID,rnk,Title,Rating_monthly_count,Review_month,Rating_monthly_avg,OverallCount,Overall_rating_avg
0,924,11,2001: A Space Odyssey (1968),24,200004,4.33,24,4.330
1,1214,10,Alien (1979),39,200004,4.36,39,4.360
2,1200,13,Aliens (1986),112,200006,4.24,287,4.160
3,2858,9,American Beauty (1999),194,200009,4.44,1556,4.366
4,2858,15,American Beauty (1999),571,200008,4.39,1362,4.356
...,...,...,...,...,...,...,...,...
85,1196,12,Star Wars: Episode V - The Empire Strikes Back...,287,200007,4.32,779,4.245
86,50,1,"Usual Suspects, The (1995)",133,200009,4.63,868,4.533
87,50,5,"Usual Suspects, The (1995)",321,200008,4.54,735,4.516
88,50,2,"Usual Suspects, The (1995)",170,200007,4.52,414,4.500


In [70]:
conn = sqlite3.connect("..//sqlite_DB//Movie_report.db")
cur = conn.cursor()
last_6_months ="""
select Month from DateDim where Month < 200009 order by Month desc limit 6
"""
last_6_months_df = pd.read_sql_query(last_6_months, conn)
conn.close()
last_6_months_df

,Month
0,200008
1,200007
2,200006
3,200005
4,200004


In [71]:
for index,row in last_6_months_df.iterrows():
    print(index,row['Month'])

0 200008
1 200007
2 200006
3 200005
4 200004


In [72]:
df_details_latest =df_details[df_details['Review_month'] =='200009'].copy()
df_details_latest.sort_values(by=['rnk'],inplace=True)

In [73]:
df_details_latest

,MovieID,rnk,Title,Rating_monthly_count,Review_month,Rating_monthly_avg,OverallCount,Overall_rating_avg
86,50,1,"Usual Suspects, The (1995)",133,200009,4.63,868,4.533
63,318,2,"Shawshank Redemption, The (1994)",130,200009,4.60,1066,4.579
57,527,3,Schindler's List (1993),122,200009,4.57,1070,4.510
38,1193,4,One Flew Over the Cuckoo's Nest (1975),98,200009,4.53,852,4.423
80,260,5,Star Wars: Episode IV - A New Hope (1977),160,200009,4.52,1446,4.470
74,2762,6,"Sixth Sense, The (1999)",148,200009,4.52,1267,4.417
49,1198,7,Raiders of the Lost Ark (1981),138,200009,4.48,1233,4.449
35,2571,8,"Matrix, The (1999)",149,200009,4.48,1257,4.336
3,2858,9,American Beauty (1999),194,200009,4.44,1556,4.366
24,858,10,"Godfather, The (1972)",117,200009,4.42,1091,4.524


In [74]:
HTML_STR ="""<HTML><HEAD>
<style>
th {background-color: powderblue;}
H1   {color: blue;}
p    {color: green;}
</style>
</HEAD><BODY>
<H1> <center> Top 10 Monthly Report </center> </H1>"""
HTML_STR = HTML_STR +"<p>The Movie Review is showing for {}</p>".format("SEP-2000") 

HTML_STR = HTML_STR+"""<TABLE border=1><TR><TH> Movie Name</TH><TH> RANK</TH><TH> Monthly Average</TH><TH> Overall Average</TH><TH> Overall Viewers</TH></TR>"""
for index,row in df_details_latest.iterrows():
    print(row['Title'])
    Movieid =row['MovieID']
    Monthly_count_diff =0
    Monthly_avg_diff =0
    overall_avg_diff =0
    rank_diff =0
    cnt =0
    for index_m,row_m in last_6_months_df.iterrows():           
        df_details_temp =df_details[(df_details['Review_month'] ==str(row_m['Month'])) & (df_details['MovieID'] ==Movieid)].copy()
        if len(df_details_temp) > 0:
            cnt =cnt +1
            if index_m ==0:
                print(row_m)
                rank_diff =int(row['rnk']) - int(df_details_temp['rnk'])
                Monthly_count_diff =int(row['Rating_monthly_count']) - int(df_details_temp['Rating_monthly_count'])
                Monthly_avg_diff =float(row['Rating_monthly_avg']) - float(df_details_temp['Rating_monthly_avg'])
                overall_avg_diff =float(row['Overall_rating_avg']) - float(df_details_temp['Overall_rating_avg'])
        else:
            break
    table_date ="<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td>".format(row['Title'],row['rnk'],row['Rating_monthly_avg'],row['Overall_rating_avg'],row['OverallCount'])
    HTML_STR =HTML_STR + table_date
HTML_STR =HTML_STR + "</TABLE></BODY></HTML>"
pdfkit.from_string(HTML_STR, "string.pdf", verbose=True)
print(HTML_STR)

Usual Suspects, The (1995)
Month    200008
Name: 0, dtype: int64
Shawshank Redemption, The (1994)
Month    200008
Name: 0, dtype: int64
Schindler's List (1993)
Month    200008
Name: 0, dtype: int64
One Flew Over the Cuckoo's Nest (1975)
Month    200008
Name: 0, dtype: int64
Star Wars: Episode IV - A New Hope (1977)
Month    200008
Name: 0, dtype: int64
Sixth Sense, The (1999)
Month    200008
Name: 0, dtype: int64
Raiders of the Lost Ark (1981)
Month    200008
Name: 0, dtype: int64
Matrix, The (1999)
American Beauty (1999)
Month    200008
Name: 0, dtype: int64
Godfather, The (1972)
Month    200008
Name: 0, dtype: int64
Princess Bride, The (1987)
Braveheart (1995)
Month    200008
Name: 0, dtype: int64
Silence of the Lambs, The (1991)
Month    200008
Name: 0, dtype: int64
Blade Runner (1982)
Fargo (1996)
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and fo

In [75]:
def getting_image(val):
    if val ==0:
        return "<img src='https://img.icons8.com/office/16/null/new.png'/>"
    elif val > 0:
        return "<img src='https://img.icons8.com/office/16/null/thick-arrow-pointing-up.png'/>"
    else:
        return "<img src='https://img.icons8.com/office/16/null/thick-arrow-pointing-down.png'/>"
          
          

In [76]:
HTML_STR ="""<HTML><HEAD>
<style>
th {background-color: powderblue;align :center}
td {text-align :center;font-size: 15px}
table {border:1px solid black}
H1   {color: blue;}
p    {color: green;}
</style>
</HEAD><BODY>
<H1> <center> Top 10 Movies Monthly Report </center> </H1>"""
HTML_STR = HTML_STR +"<p>The Movie Review is showing for {}</p>".format("Sep-2000") 

HTML_STR = HTML_STR+"""<TABLE><TR><TH> Movie Name</TH><TH colspan=3> RANK</TH><TH>Consequtive week</TH><TH colspan=3> Monthly Average</TH><TH colspan=3> Overall Average</TH><TH> Overall Viewers</TH></TR>"""
for index,row in df_details_latest.iterrows():
    print(row['Title'])
    Movieid =row['MovieID']
    Monthly_count_diff =0
    Monthly_avg_diff =0
    overall_avg_diff =0
    rank_diff =0
    cnt =0
    for index_m,row_m in last_6_months_df.iterrows():           
        df_details_temp =df_details[(df_details['Review_month'] ==str(row_m['Month'])) & (df_details['MovieID'] ==Movieid)].copy()
        if len(df_details_temp) > 0:
            cnt =cnt +1
            if index_m ==0:
                rank_diff =  int(df_details_temp['rnk']) -int(row['rnk'])
                Monthly_count_diff =int(row['Rating_monthly_count']) - int(df_details_temp['Rating_monthly_count'])
                Monthly_avg_diff =float(row['Rating_monthly_avg']) - float(df_details_temp['Rating_monthly_avg'])
                overall_avg_diff =float(row['Overall_rating_avg']) - float(df_details_temp['Overall_rating_avg'])
        else:
            break
    table_date ="<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(row['Title'],row['rnk'],getting_image(rank_diff),rank_diff,cnt,row['Rating_monthly_avg'],getting_image(Monthly_avg_diff),round(Monthly_avg_diff,3),row['Overall_rating_avg'],getting_image(overall_avg_diff),round(overall_avg_diff,3),row['OverallCount'])
    HTML_STR =HTML_STR + table_date
HTML_STR =HTML_STR + "</TABLE></BODY></HTML>"
pdfkit.from_string(HTML_STR, "string.pdf", verbose=True)
print(HTML_STR)

Usual Suspects, The (1995)
Shawshank Redemption, The (1994)
Schindler's List (1993)
One Flew Over the Cuckoo's Nest (1975)
Star Wars: Episode IV - A New Hope (1977)
Sixth Sense, The (1999)
Raiders of the Lost Ark (1981)
Matrix, The (1999)
American Beauty (1999)
Godfather, The (1972)
Princess Bride, The (1987)
Braveheart (1995)
Silence of the Lambs, The (1991)
Blade Runner (1982)
Fargo (1996)
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
<HTML><HEAD>
<style>
th {background-color: powderblue;align :center}
td {text-align :center;font-size: 15px}
table {border:1px solid black}
H1   {color: blue;}
p    {color: green;}
</style>
</HEAD><BODY>
<H1> <center> Top 10 Movies Monthly Report </center> </H1><p>Th